In [54]:
import spacy
import coreferee
from spacy.matcher import Matcher
import pandas as pd
import pickle
from spacy import displacy


In [2]:
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 3200000
nlp.add_pipe('coreferee')


In [3]:
text = ""
with open("./LOTR_COREF.txt", encoding="utf8") as f:
  try:
    text = f.readlines()
  except Exception as e:
    print(e)


In [6]:
import re
string_text = ""
for line in text:
  string_text += line + "\n"
string_text = re.sub('[\n]+', '\n', string_text)


In [23]:

sentences = string_text.split(".")
new_sentences = []
for sentence in sentences:
  splitted = sentence.split(",")
  for s in splitted:
    new_sentences.append(s)
new_sentences[2]

' there was much talk and excitement in Hobbiton '

In [28]:
def get_entities(sent):

  entity1 = ""
  entity2 = ""

  prev_token_dependency = ""    
  prev_token_text = ""   

  prefix = ""
  modifier = "" 
  for tok in nlp(sent):
    if tok.dep_ != "punct":
      if tok.dep_ == "compound":
        if prev_token_dependency == "compound":
          prefix = prev_token_text + " " + tok.text
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        if prev_token_dependency == "compound":
          modifier = prev_token_text + " "+ tok.text
      if tok.dep_.find("subj") == True:
        entity1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prev_token_dependency = ""
        prev_token_text = ""
      if tok.dep_.find("obj") == True:
        entity2 = modifier + " " + prefix + " " + tok.text     
      prev_token_dependency = tok.dep_
      prev_token_text = tok.text
  return [entity1.strip(), entity2.strip()]


In [25]:
def extract_entities_from_sentence(sentence):
  doc = nlp(sentence)
  previous_token_dependency = ""
  previous_token_text = ""

  entita1 = ""
  entita2 = ""

  prefisso = ""
  modificatore = ""
  for token in doc:
    if token.dep_ != "punct":
      if token.dep_ == "compound":
        prefisso = token.text
        if previous_token_dependency == "compound":
          prefisso = previous_token_text + " " + token.text
      if token.dep_.endswith("mod"):
        modificatore = token.text
        if previous_token_dependency == "compound":
          modificatore = previous_token_text + " " + token.text
      if token.dep_.find("subj"):
        entita1 = modificatore+" "+prefisso+" "+token.text
        prefisso=""
        modificatore=""
        previous_token_dependency = ""
        previous_token_text = ""
      if token.dep_.find("obj"):
        entita2 = modificatore+" "+prefisso+" "+token.text
      previous_token_dependency = token.dep_
      previous_token_text = token.text
  return [entita1, entita2]
       

In [46]:
def extract_relations(sentence):
  doc = nlp(sentence)
  matcher = Matcher(nlp.vocab) 
  pattern = [{'DEP': 'ROOT'},
             {'DEP': 'prep', 'OP': "?"},
             {'DEP': 'agent', 'OP': "?"},
             {'POS': 'ADJ', 'OP': "?"}]
  matcher.add("matching_1", [pattern])
  matches = matcher(doc)
  match_index = len(matches)-1
  text = doc[matches[match_index][1]:matches[match_index][2]].text
  return text


In [36]:
new_sentences[5]

' ever since Bilbo remarkable disappearance and unexpected return '

In [2]:
import opennre

model = opennre.get_model('wiki80_cnn_softmax')


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [45]:
index = 12
print(get_entities(new_sentences[index]))
extract_relations(new_sentences[index])


['Time', '']
wore


In [55]:
document = nlp(new_sentences[1])
displacy.serve(document, style="dep")


c:\Users\Ruben\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\displacy\__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [51]:
triples = []
for sentence in new_sentences:
  try:
    entities = get_entities(sentence)
    relation = extract_relations(sentence)
    if len(entities) ==2 and relation and entities[0] != '' and entities[1] != '':
      triples.append([entities[0], relation, entities[1]])
  except Exception as e:
    pass

In [52]:
triples[:10]

[['Bag Baggins', 'announced', 'special  magnificence'],
 ['Bilbo', 'become', 'back Bilbo travels'],
 ['well  unchanged', 'began', 'mark'],
 ['anyone', 'seemed unfair', 'perpetual  youth'],
 ['far  trouble', 'come', 'Mr'],
 ['Baggins', 'was generous', 'Baggins money'],
 ['most  people', 'were willing', 'Baggins Baggins oddities'],
 ['Baggins', 'remained on', 'visiting Baggins relatives'],
 ['Baggins', 'had', 'close  friends'],
 ['ninety nine ninety Bilbo', 'adopted', 'Bilbo heir']]

In [53]:
len(triples)

7801

In [20]:
sentences[2]

' \n Bilbo was very rich and very peculiar , and had been the wonder of the Shire for sixty years , ever since Bilbo remarkable disappearance and unexpected return '